<a href="https://colab.research.google.com/github/huggingface/deep-rl-class/blob/main/notebooks/unit6/unit6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="In Colab öffnen"/></a>

# Einheit 6: Advantage Actor Critic (A2C) unter Verwendung von Robotik-Simulationen mit Panda-Gym 🤖

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/thumbnail.png" alt="Thumbnail"/>

In diesem Notizbuch lernst du, A2C mit [Panda-Gym](https://github.com/qgallouedec/panda-gym) zu verwenden. Sie werden **einen Roboterarm** (Franka Emika Panda Roboter) trainieren, um eine Aufgabe auszuführen:

- Reichweite": Der Roboter muss seinen Endeffektor an einer Zielposition platzieren.

Danach können Sie **andere Robotik-Aufgaben** trainieren.


### 🎮 Umgebungen:

- [Panda-Gym](https://github.com/qgallouedec/panda-gym)

###📚 RL-Library:

- [Stable-Baselines3](https://stable-baselines3.readthedocs.io/)

Wir versuchen ständig, unsere Tutorials zu verbessern, also **wenn Sie Probleme in diesem Notizbuch** finden, öffnen Sie bitte [einen Fehler im GitHub Repo](https://github.com/huggingface/deep-rl-class/issues).

## Ziele dieses Notizbuchs 🏆

Am Ende des Notizbuchs werden Sie:

- In der Lage sein, **Panda-Gym**, die Umgebungsbibliothek, zu benutzen.
- In der Lage sein, **Roboter mit A2C** zu trainieren.
- Verstehen, warum **wir die Eingabe normalisieren müssen**.
- In der Lage sein, **Ihren trainierten Agenten und den Code auf den Hub zu pushen**, mit einer schönen Videowiedergabe und einer Bewertungsnote 🔥.




## Dieses Notizbuch stammt aus dem Deep Reinforcement Learning Kurs.
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

In diesem kostenlosen Kurs werden Sie:

- 📖 Deep Reinforcement Learning in **Theorie und Praxis** studieren.
- 🧑‍💻 Lernen Sie, **berühmte Deep RL-Bibliotheken** wie Stable Baselines3, RL Baselines3 Zoo, CleanRL und Sample Factory 2.0 zu verwenden.
- 🤖 Trainieren Sie **Agenten in einzigartigen Umgebungen**.

Und mehr, siehe 📚 den Lehrplan 👉 https://simoninithomas.github.io/deep-rl-course

Vergessen Sie nicht, sich **<a href="http://eepurl.com/ic5ZUD">für den Kurs anzumelden</a>** (wir sammeln Ihre E-Mail, um Ihnen **die Links zu senden, wenn die einzelnen Einheiten veröffentlicht werden, und Sie über die Herausforderungen und Aktualisierungen zu informieren).**


Der beste Weg, um in Kontakt zu bleiben, ist, unserem Discord-Server beizutreten, um sich mit der Community und mit uns auszutauschen 👉🏻 https://discord.gg/ydHrjt3WP5

## Voraussetzungen 🏗️
Bevor Sie sich mit dem Notebook beschäftigen, müssen Sie:

🔲 📚 Studium der [akteurskritischen Methoden durch Lektüre von Einheit 6](https://huggingface.co/deep-rl-course/unit6/introduction) 🤗

# Lasst uns unsere ersten Roboter trainieren 🤖.

Um dieses Hands-on für den [Zertifizierungsprozess] (https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process) zu validieren, müssen Sie Ihr trainiertes Modell an den Hub senden und die folgenden Ergebnisse erhalten:

- PandaReachDense-v3" erhält ein Ergebnis von >= -3,5.

Um Ihr Ergebnis zu finden, gehen Sie zur [Bestenliste] (https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) und suchen Sie Ihr Modell, **das Ergebnis = mean_reward - std of reward**

Weitere Informationen über den Zertifizierungsprozess finden Sie in diesem Abschnitt 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## Die GPU einstellen 💪.
- Um **das Training des Agenten zu beschleunigen, werden wir einen Grafikprozessor** verwenden. Gehen Sie dazu auf "Runtime > Change Runtime type".

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Schritt 1">

- Hardware-Beschleuniger > GPU".

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Schritt 2">

## Erstellen einer virtuellen Anzeige 🔽.

Während der Arbeit mit dem Notebook müssen wir ein Wiederholungsvideo erstellen. Dazu benötigen wir mit colab **einen virtuellen Bildschirm, um die Umgebung zu rendern** (und somit die Bilder aufzunehmen).

Daher wird die folgende Zelle die Librairies installieren und einen virtuellen Bildschirm erstellen und starten 🖥

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

### Abhängigkeiten installieren 🔽.

Der erste Schritt ist die Installation der Abhängigkeiten, wir werden mehrere davon installieren:
- `gymnasium`
- panda-gym": Enthält die Umgebungen für die Roboterarme.
- stable-baselines3": Die SB3-Bibliothek für Deep Reinforcement Learning.
- Umarmungsfläche_sb3`: Zusätzlicher Code für Stable-baselines3 zum Laden und Hochladen von Modellen aus dem Hugging Face 🤗 Hub.
- huggingface_hub`: Bibliothek, die es jedem erlaubt, mit den Hub-Repositories zu arbeiten.

⏲ Die Installation kann **10 Minuten** dauern.

In [ ]:
!pip install stable-baselines3[extra]
!pip install gymnasium

In [ ]:
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

## Importieren Sie die Pakete 📦

In [ ]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

## PandaReachDense-v3 🦾

Der Agent, den wir trainieren wollen, ist ein Roboterarm, der Steuerungen durchführen muss (den Arm bewegen und den Endeffektor benutzen).

In der Robotik ist der *Endeffektor* das Gerät am Ende eines Roboterarms, das für die Interaktion mit der Umgebung bestimmt ist.

In `PandaReach` muss der Roboter seinen Endeffektor an einer Zielposition (grüner Ball) platzieren.

Wir werden die dichte Version dieser Umgebung verwenden. Das bedeutet, dass wir eine *dichte Belohnungsfunktion* erhalten, die **bei jedem Zeitschritt** eine Belohnung bereitstellt (je näher der Agent an der Erfüllung der Aufgabe ist, desto höher die Belohnung). Im Gegensatz zu einer *sparsamen Belohnungsfunktion*, bei der die Umgebung **eine Belohnung zurückgibt, wenn und nur wenn die Aufgabe erfüllt ist**.

Außerdem verwenden wir die *End-Effektor-Verschiebungssteuerung*, d.h. die **Aktion entspricht der Verschiebung des Endeffektors**. Wir steuern nicht die individuelle Bewegung jedes Gelenks (Gelenksteuerung).

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/robotics.jpg" alt="Robotik"/>


Auf diese Weise wird **das Training einfacher**.



### Erstellen Sie die Umgebung

#### Die Umgebung 🎮

In `PandaReachDense-v3` muss der Roboterarm seinen Endeffektor an einer Zielposition (grüne Kugel) platzieren.

In [ ]:
env_id = "PandaReachDense-v3"

# Create the env
env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape
a_size = env.action_space

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

Der Beobachtungsraum **ist ein Wörterbuch mit 3 verschiedenen Elementen**:
- `erreichtes_Ziel`: (x,y,z) Position des Ziels.
- angestrebtes_Ziel`: (x,y,z) Abstand zwischen der Zielposition und der aktuellen Objektposition.
- Beobachtung": Position (x,y,z) und Geschwindigkeit des Endeffektors (vx, vy, vz).

Da es sich bei der Beobachtung um ein Wörterbuch handelt, **müssen wir eine MultiInputPolicy-Richtlinie anstelle von MlpPolicy** verwenden.

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

Der Aktionsraum ist ein Vektor mit 3 Werten:
- Steuerung x, y, z Bewegung

### Beobachtung und Belohnung normalisieren

Eine gute Praxis beim Reinforcement Learning ist die [Normalisierung der Input-Features] (https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html).

Zu diesem Zweck gibt es einen Wrapper, der einen laufenden Durchschnitt und eine Standardabweichung der Eingangsmerkmale berechnet.

Wir normalisieren auch die Belohnungen mit demselben Wrapper, indem wir `norm_reward = True` hinzufügen

[Sie sollten die Dokumentation lesen, um diese Zelle zu füllen](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecnormalize)

In [ ]:
env = make_vec_env(env_id, n_envs=4)

# Adding this wrapper to normalize the observation and the reward
env = # TODO: Add the wrapper

#### Lösung

In [ ]:
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

### Erstellen des A2C-Modells 🤖

Weitere Informationen zur A2C-Implementierung mit StableBaselines3 finden Sie unter: https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html#notes

Um die besten Parameter zu finden, habe ich die [offiziellen geschulten Agenten des Stable-Baselines3-Teams] (https://huggingface.co/sb3) geprüft.

In [ ]:
model = # Create the A2C model and try to find the best parameters

#### Lösung

In [ ]:
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)

### Trainieren Sie den A2C-Agenten 🏃
- Lassen Sie uns unseren Agenten für 1.000.000 Zeitschritte trainieren, vergessen Sie nicht, die GPU auf Colab zu verwenden. Es wird ungefähr ~25-40min dauern

In [ ]:
model.learn(1_000_000)

In [ ]:
# Save the model and  VecNormalize statistics when saving the agent
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

### Bewerten Sie den Agenten 📈
- Nun, da unser Agent trainiert ist, müssen wir seine Leistung **prüfen**.
- Stable-Baselines3 bietet dafür eine Methode: `evaluate_policy`

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# We need to override the render_mode
eval_env.render_mode = "rgb_array"

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-PandaReachDense-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

### Veröffentlichen Sie Ihr trainiertes Modell auf dem Hub 🔥
Da wir nun gesehen haben, dass wir nach dem Training gute Ergebnisse erzielt haben, können wir unser trainiertes Modell mit einer Zeile Code auf dem Hub veröffentlichen.

📚 Die Dokumentation der Bibliotheken 👉 https://github.com/huggingface/huggingface_sb3/tree/main#hugging-face--x-stable-baselines3-v20


Durch die Verwendung von `package_to_hub`, wie wir bereits in den früheren Einheiten erwähnt haben, **werten Sie aus, zeichnen eine Wiedergabe auf, erzeugen eine Modellkarte Ihres Agenten und schieben sie zum Hub**.

This way:
- Du kannst **unsere Arbeit vorführen** 🔥.
- Sie können **Ihren Agenten beim Spielen visualisieren** 👀
- Du kannst **einen Agenten mit der Community teilen, den andere benutzen können** 💾
- Sie können **auf eine Bestenliste 🏆 zugreifen, um zu sehen, wie gut Ihr Agent im Vergleich zu Ihren Klassenkameraden abschneidet** 👉 https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard


Um Ihr Modell mit der Gemeinschaft teilen zu können, sind drei weitere Schritte erforderlich:

1️⃣ (Falls noch nicht geschehen) Erstellen Sie ein Konto für HF ➡ https://huggingface.co/join

2️⃣ Melde dich an und speichere dann dein Authentifizierungs-Token von der Hugging Face Website.
- Erstellen Sie ein neues Token (https://huggingface.co/settings/tokens) **mit Schreibrolle**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="HF-Token erstellen">

- Kopieren Sie das Token
- Führen Sie die Zelle unten aus und fügen Sie das Token ein

In [ ]:
notebook_login()
!git config --global credential.helper store

Wenn Sie kein Google Colab oder ein Jupyter Notebook verwenden möchten, müssen Sie stattdessen diesen Befehl verwenden: `huggingface-cli login`

3️⃣ Wir sind jetzt bereit, unseren trainierten Agenten mit der Funktion `package_to_hub()` an den 🤗 Hub 🔥 zu übertragen.

In dieser Umgebung kann die **Ausführung dieser Zelle etwa 10 Minuten dauern**.

In [ ]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"ThomasSimonini/a2c-{env_id}", # Change the username
    commit_message="Initial commit",
)

## Einige zusätzliche Herausforderungen 🏆
Der beste Weg zu lernen **ist, Dinge selbst auszuprobieren**! Warum nicht `PandaPickAndPlace-v3` ausprobieren?

Wenn du fortgeschrittenere Aufgaben für panda-gym ausprobieren willst, musst du prüfen, was mit **TQC oder SAC** (einem für Robotikaufgaben geeigneten, stichprobenartigeren Algorithmus) gemacht wurde. In der realen Robotik werden Sie aus einem einfachen Grund einen Algorithmus verwenden, der effizienter ist: Im Gegensatz zu einer Simulation **wenn Sie Ihren Roboterarm zu stark bewegen, besteht die Gefahr, dass er kaputt geht**.

PandaPickAndPlace-v1 (dieses Modell verwendet die v1-Version der Umgebung): https://huggingface.co/sb3/tqc-PandaPickAndPlace-v1

Und zögern Sie nicht, die Panda-Gym-Dokumentation hier zu lesen: https://panda-gym.readthedocs.io/en/latest/usage/train_with_sb3.html

Wir stellen Ihnen die Schritte zum Trainieren eines weiteren Agenten zur Verfügung (optional):

1. Definieren Sie eine Umgebung namens "PandaPickAndPlace-v3".
2. Erstellen Sie eine vektorisierte Umgebung
3. Füge einen Wrapper hinzu, um die Beobachtungen und Belohnungen zu normalisieren. [Prüfen Sie die Dokumentation](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecnormalize)
4. Erstellen Sie das A2C-Modell (vergessen Sie nicht verbose=1, um die Trainingsprotokolle zu drucken).
5. Trainieren Sie es für 1M Timesteps
6. Speichern Sie das Modell und VecNormalize-Statistiken beim Speichern des Agenten
7. Beurteilen Sie Ihren Agenten
8. Veröffentliche dein trainiertes Modell auf dem Hub 🔥 mit `package_to_hub`.


### Lösung (optional)

In [ ]:
# 1 - 2
env_id = "PandaPickAndPlace-v3"
env = make_vec_env(env_id, n_envs=4)

# 3
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

# 4
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)
# 5
model.learn(1_000_000)

In [ ]:
# 6
model_name = "a2c-PandaPickAndPlace-v3";
model.save(model_name)
env.save("vec_normalize.pkl")

# 7
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("PandaPickAndPlace-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load(model_name)

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

# 8
package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"ThomasSimonini/a2c-{env_id}", # TODO: Change the username
    commit_message="Initial commit",
)

Wir sehen uns in Referat 7! 🔥
## Lernt weiter, bleibt fantastisch 🤗